# Seeds exploration

The idea is to expand the set of words of interest; in fact, not all the suggested words are present in the corpus, and some of them have really low frequencies. This is a problem when trying to guide to topic modelling around the concepts of interests, because these words appear in few documents. Trying to expand the set should help us in discovering better topics (and in general in doing a better analysis).

In [7]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("..")

from pprint import pprint
import itertools

from src.dataset import Dataset

import spacy
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner']) # just keep tagger for lemmatization

In [8]:
dataset = Dataset()
tokens = dataset.load_dataset(year=None, 
                              tokens=True, 
                              courts={"Illinois Supreme Court"})

In [18]:
from collections import defaultdict
def check_corpus_presence(narcotics, weapons, investigation, filtered_tokens, topN=100, notFoundN=100):
    seen = set()
    not_found = weapons.union(investigation).union(narcotics)
    freq = defaultdict(lambda:0)
    for doc in filtered_tokens:
        for w in set(doc): 
            if w in not_found:
                seen.add(w)
                freq[w] += 1
    
    not_found -= seen
                
    sorted_words = sorted([(v,k) for k,v in freq.items()])
    if topN:
        print(', '.join([str(t) for t in sorted_words[:topN]]))
    else:
        print(', '.join([str(t) for t in sorted_words]))    
    print(f"*** Not found ({len(not_found)} words): ***")
    if notFoundN:
        print(', '.join([t for t in itertools.islice(not_found, notFoundN)]))
    else:
        print(', '.join(not_found))    
    
    return sorted_words, not_found

### Initial sets
We keep singular and plural versions of words because the pre-processing phase can produce both versions, depending on the context (we use Spacy)

In [19]:
narcotics = {'cannabis', 'cocaine', 'methamphetamine', 'drugs', 'drug', 'marijuana', 
             'ecstasy', 'lsd', 'ketamine', 'heroin', 'fentanyl', 'overdose', 'blunt'}

weapons = {'gun', 'knife', 'weapon', 'firearm', 'rifle', 'carbine', 'shotgun', 'handgun', 
           'revolver', 'musket', 'pistol', 'derringer', 'assault', 'rifle', 'sword'}

investigation = {'gang', 'mafia', 'serial',  'killer', 'rape', 'theft', 'recidivism', 
                 'arrest', 'robbery', 'cybercrime', 'cyber', 'crime'}

As we can see, many words don't appear so frequently in the collection, and some words don't even appear; if we want to try to drive the Topic Modelling process towards topics of interest, we need to expand these sets with more frequent words.

In [20]:
_ = check_corpus_presence(narcotics, weapons, investigation, tokens)

(1, 'cyber'), (1, 'ketamine'), (2, 'musket'), (5, 'mafia'), (8, 'derringer'), (14, 'drugs'), (18, 'methamphetamine'), (20, 'carbine'), (20, 'overdose'), (29, 'lsd'), (32, 'recidivism'), (100, 'sword'), (138, 'killer'), (142, 'cannabis'), (158, 'serial'), (203, 'blunt'), (204, 'handgun'), (205, 'rifle'), (265, 'shotgun'), (288, 'marijuana'), (290, 'heroin'), (320, 'cocaine'), (359, 'firearm'), (392, 'gang'), (721, 'pistol'), (773, 'knife'), (923, 'revolver'), (975, 'theft'), (1043, 'rape'), (1475, 'weapon'), (1611, 'drug'), (1941, 'gun'), (2053, 'assault'), (2279, 'robbery'), (5971, 'arrest'), (6662, 'crime')
*** Not found (3 words): ***
fentanyl, ecstasy, cybercrime


In [21]:
not_found_in_collection = {'ecstasy', 'cybercrime', 'fentanyl'}

interesting_set = narcotics.union(weapons).union(investigation)

## GoogleNews word embeddings

Using word embeddings, it should be possible to find the words that are used in the same context as our words of interest, so that we can expand the seed set with different words that are used in the same context.

In [22]:
from gensim import models

w = models.KeyedVectors.load_word2vec_format(
    '../data/models/GoogleNews-vectors-negative300.bin', binary=True)

In [23]:
w.most_similar(positive=['cocaine'], topn=5)

[('heroin', 0.8294118046760559),
 ('crack_cocaine', 0.8008098006248474),
 ('methamphetamine', 0.7232441306114197),
 ('narcotics', 0.707099974155426),
 ('methamphetamines', 0.7007291316986084)]

In [24]:
top_words = 5

for word in interesting_set:
    if word == "blunt": # usually blunt is too generic, not referring to drugs
        similar_words = w.most_similar(positive=[word, "drug", "marijuana"], topn=top_words)
    elif word == "serial": # too generic by itself
        similar_words = w.most_similar(positive=[word, "killer"], topn=top_words)    
    else:
        similar_words = w.most_similar(positive=[word], topn=top_words)
    print(f"*** {word} ***:\n {' - '.join(map(lambda x: f'{x[0]}, {round(x[1], 2)}', similar_words))}")

*** pistol ***:
 rifle, 0.84 - pistols, 0.8 - handgun, 0.79 - 9mm_pistol, 0.77 - gun, 0.77
*** gun ***:
 handgun, 0.81 - guns, 0.78 - pistol, 0.77 - firearm, 0.74 - firearms, 0.73
*** drug ***:
 drugs, 0.85 - Drug, 0.75 - narcotics, 0.64 - cocaine, 0.61 - heroin, 0.61
*** heroin ***:
 cocaine, 0.83 - Heroin, 0.77 - crack_cocaine, 0.74 - narcotics, 0.73 - methamphetamine, 0.72
*** firearm ***:
 firearms, 0.8 - handgun, 0.77 - gun, 0.74 - possessing_firearm, 0.71 - pistol, 0.7
*** carbine ***:
 rifle, 0.71 - carbines, 0.69 - rifles, 0.65 - M4_carbine, 0.64 - pistol, 0.63
*** killer ***:
 killers, 0.76 - murderer, 0.66 - Killer, 0.64 - serial_killer, 0.6 - rapist, 0.57
*** methamphetamine ***:
 meth, 0.9 - methamphetamines, 0.86 - crystal_methamphetamine, 0.8 - Methamphetamine, 0.79 - meth_amphetamine, 0.76
*** fentanyl ***:
 Fentanyl, 0.72 - oxycodone, 0.66 - painkiller_fentanyl, 0.64 - fentanyl_patch, 0.62 - morphine, 0.62
*** sword ***:
 swords, 0.77 - broadsword, 0.66 - sandal_flick, 

Some words are compounds words, which would be separated by a whitespace but in order to keep them in googleNewsVectors it has been chosen to use the underscore in order to combine them; we need to split them in singular words, because we are expanding the set using only single words.

In [25]:
top_words = 100

def addWordsToSet(wordSet, similar_words, lemmatize=False):
    if lemmatize:
        for similar_word in similar_words:
             words = [" ".join([token.lemma_ for token in nlp(similar.lower())]) for similar in similar_word[0].split("_")]  # split composite words in singular ones
             wordSet.update(words)
    else:
        for similar_word in similar_words:
            wordSet.update([w.lower() for w in similar_word[0].split("_")])


### Start expansion process
Get top n words for each word of interest. This is our starting point in order to expand the seeds.

In [26]:
new_narcotics, new_weapons, new_investigation = set(), set(), set()
for word in interesting_set:
    if word == "blunt": # usually blunt is too generic, not referring to drugs
        similar_words = w.most_similar(positive=[word, "drug", "marijuana"], topn=top_words)
    elif word == "serial": # too generic by itself
        similar_words = w.most_similar(positive=[word, "killer"], topn=top_words)
    else:
        similar_words = w.most_similar(positive=[word], topn=top_words)
        
    if word in narcotics:
        addWordsToSet(new_narcotics, similar_words)
    elif word in weapons:
        addWordsToSet(new_weapons, similar_words)
    elif word in investigation:
        addWordsToSet(new_investigation, similar_words)
    
print(f"*** New narcotics ({len(new_narcotics)} words): ***\n{', '.join([t for t in itertools.islice(new_narcotics, 50)])}")
print(f"*** New weapons ({len(new_weapons)} words): ***\n{', '.join([t for t in itertools.islice(new_weapons, 50)])}")
print(f"*** New investigation ({len(new_investigation)} words) ***:\n{', '.join([t for t in itertools.islice(new_investigation, 50)])}")

*** New narcotics (496 words): ***
anesthetic, antipsychotic, amphetamine, suspected, coroner, lortab, pgi2, smokable, methyl, ecstacy, imitation, acetylcysteine, yerba, cardiac, toprol, hallucinogens, deoxy, reservation, euphoric, mdma, muscle, trafficable, paraphenalia, methampetamine, adipex, amytal, commonly, subutex, dagga, powder, vertex, klonopin, adderal, amyl, od'd, benzylpiperazine, psychostimulant, fentanyl, suboxone, toxicity, medicines, syabu, pneumonia, glibenclamide, hydroponically, ghb, cialis, arrest, heart, marajuana
*** New weapons (485 words): ***
guns, scimitar, wielding, longbows, plated, hammer, mallet, ammunition, bows, cannonball, occasioning, gravity, imitation, assult, airsoft, &, semiautomatic, chambering, lightsabres, nine, iiia, unlicenced, degree, assualting, damocles, article, indecent, kukri, warrior, wesson, scepter, paring, gage, volleys, cleaver, enactments, dragoons, vicious, cobra, powder, enactor, lascivious, .9, short, gladius, silencer, steak, h

In [27]:
found_google, not_found_google = check_corpus_presence(new_narcotics, new_weapons, new_investigation, tokens)

(1, 'absconder'), (1, 'aces'), (1, 'affiliations'), (1, 'ambien'), (1, 'amitriptyline'), (1, 'amphetamines'), (1, 'antidepression'), (1, 'arrrest'), (1, 'assualt'), (1, 'assult'), (1, 'attacks'), (1, 'bali'), (1, 'balls'), (1, 'boots'), (1, 'botched'), (1, 'bredel'), (1, 'bullying'), (1, 'calabrian'), (1, 'chainsaw'), (1, 'chucks'), (1, 'chucky'), (1, 'cleared'), (1, 'clonazepam'), (1, 'concealed'), (1, 'cooks'), (1, 'criminals'), (1, 'crip'), (1, 'crooks'), (1, 'cyber'), (1, 'dextromethorphan'), (1, 'diazepam'), (1, 'effects'), (1, 'epic'), (1, 'familia'), (1, 'felons'), (1, 'flintlock'), (1, 'foxen'), (1, 'genovese'), (1, 'grasso'), (1, 'harasser'), (1, 'hcl'), (1, 'herion'), (1, 'hilltop'), (1, 'homocide'), (1, 'howitzer'), (1, 'id'), (1, 'indictments'), (1, 'insider'), (1, 'ketamine'), (1, 'kiddie'), (1, 'killings'), (1, 'krishna'), (1, 'langner'), (1, 'larcency'), (1, 'legalizing'), (1, 'lorazepam'), (1, 'manhunt'), (1, 'meow'), (1, 'mice'), (1, 'milly'), (1, 'minie'), (1, 'minimi

Not all the words of googleNews are present in our collection, also because we use Spacy for the lemmatization during the pre-processing phase. 
While Spacy shouldn't be used to lemmatize singular words, it should still be better to use it rather than considering directly the words without any kind of pre-processing.

In [28]:
new_narcotics_lemma, new_weapons_lemma, new_investigation_lemma = set(), set(), set()
for word in interesting_set:
    if word == "blunt": # usually blunt is too generic, not referring to drugs
        similar_words = w.most_similar(positive=[word, "drug", "marijuana"], topn=top_words)
    elif word == "serial": # too generic by itself
        similar_words = w.most_similar(positive=[word, "killer"], topn=top_words)
    else:
        similar_words = w.most_similar(positive=[word], topn=top_words)
        
    if word in narcotics:
        addWordsToSet(new_narcotics_lemma, similar_words, True)
    elif word in weapons:
        addWordsToSet(new_weapons_lemma, similar_words, True)
    elif word in investigation:
        addWordsToSet(new_investigation_lemma, similar_words, True)
    
print(f"*** New narcotics ({len(new_narcotics_lemma)} words): ***\n{', '.join([t for t in itertools.islice(new_narcotics_lemma, 50)])}")
print(f"*** New weapons ({len(new_weapons_lemma)} words): ***\n{', '.join([t for t in itertools.islice(new_weapons_lemma, 50)])}")
print(f"*** New investigation ({len(new_investigation_lemma)} words) ***:\n{', '.join([t for t in itertools.islice(new_investigation_lemma, 50)])}")


*** New narcotics (462 words): ***
anesthetic, drown, antipsychotic, prescribe, amphetamine, lortab, coroner, lace, pgi2, smokable, methyl, ecstacy, imitation, acetylcysteine, yerba, cardiac, toprol, deoxy, reservation, euphoric, mdma, muscle, trafficable, paraphenalia, methampetamine, adipex, amytal, apcali, commonly, subutex, dagga, 5 mg, powder, vertex, mouse, adderal, klonopin, amyl, od'd, benzylpiperazine, psychostimulant, fentanyl, suboxone, toxicity, medicine, syabu, pneumonia, glibenclamide, hydroponically, ghb
*** New weapons (428 words): ***
scimitar, hammer, mallet, ammunition, cannonball, gravity, imitation, ar # #, unlicence, assult, airsoft, &, semiautomatic, nine, iiia, degree, article, curve, indecent, kukri, warrior, wesson, scepter, gage, holstere, cleaver, grab, vicious, cobra, powder, enactor, lascivious, .9, short, gladius, silencer, steak, flash, breastplate, m, boxcutter, retractable, machete, mauser, a. # #, # .##x##mm, ball, unlawful, springfield, arrest
*** Ne

In [29]:
found_google_lemma, not_found_google_lemma = check_corpus_presence(new_narcotics_lemma, new_weapons_lemma, new_investigation_lemma, tokens)

(1, 'absconder'), (1, 'ambien'), (1, 'amitriptyline'), (1, 'antidepression'), (1, 'antiretroviral'), (1, 'arrrest'), (1, 'assualt'), (1, 'assult'), (1, 'bali'), (1, 'batterer'), (1, 'blunderbuss'), (1, 'breastplate'), (1, 'bredel'), (1, 'bugle'), (1, 'bungle'), (1, 'calabrian'), (1, 'chainsaw'), (1, 'chucky'), (1, 'clonazepam'), (1, 'crip'), (1, 'cyber'), (1, 'dextromethorphan'), (1, 'diazepam'), (1, 'dug'), (1, 'epic'), (1, 'familia'), (1, 'flintlock'), (1, 'foxen'), (1, 'gangbanger'), (1, 'gatle'), (1, 'genovese'), (1, 'grasso'), (1, 'harasser'), (1, 'hcl'), (1, 'herion'), (1, 'hilltop'), (1, 'homocide'), (1, 'howitzer'), (1, 'insider'), (1, 'ketamine'), (1, 'kiddie'), (1, 'krishna'), (1, 'langner'), (1, 'larcency'), (1, 'lorazepam'), (1, 'manhunt'), (1, 'meow'), (1, 'milly'), (1, 'minie'), (1, 'minimi'), (1, 'mmj'), (1, 'molestor'), (1, 'neonaticide'), (1, 'nonlethal'), (1, 'norinco'), (1, 'nostra'), (1, 'painkiller'), (1, 'paxil'), (1, 'phenytoin'), (1, 'polydrug'), (1, 'profiler')

In [30]:
never_found = not_found_google.union(not_found_google_lemma)
print(f"*** Never found ({len(never_found)} words): ***\n{', '.join([t for t in itertools.islice(never_found, 100)])}")

*** Never found (903 words): ***
scimitar, heldreth, occasioning, smokable, adware, thriller, wars, ar # #, dr ., mdma, &, schoolgirl, lightsabres, methampetamine, sureno, kukri, ddos, stolen, cyberdefense, backpacker, subutex, dagga, aarelaid, 5 mg, klonopin, adderal, hacktivism, benzylpiperazine, gladius, hangs, pceu, infowar, psychostimulant, burglarized, suboxone, mauser, hydroponically, ghb, .##-###, chiques, assaults, cybersecurity, croxteth, piperazine, endocet, xanex, crocky, m##a #, dhananjoy, mushrooms, nunchuks, stickups, decriminalizing, phishers, cyberspie, arrested, suppressor, chambered, injecting, propecia, m # #, evron, whodunnit, vandalisms, ar##, m9, wasarreste, crime.the, worms, opiods, trafficante, cyberthreats, cells, painkille, promethazine, vatos, lod, attemped, percocet, cocain, benzodiazepine, botmasters, xr, mafioso, filleting, combating, §, 5, lightsabre, chews, barons, breadknife, hydroxybutyrate, proxamol, alprazolam, vato, hourlong, pawan, i d, cyberterro

### Combine and filter results
We have both the words and their lemmas, we unite them and we subtract from the resulting sets the words that don't appear in our collection.

In [31]:
expanded_narcotics = (set(narcotics).union(new_narcotics).union(new_narcotics_lemma)) - not_found_google - not_found_google_lemma
expanded_weapons = (set(weapons).union(new_weapons).union(new_weapons_lemma)) - not_found_google - not_found_google_lemma
expanded_investigation = (set(investigation).union(new_investigation).union(new_investigation_lemma)) - not_found_google - not_found_google_lemma

In [32]:
print(f"*** Expanded narcotics ({len(expanded_narcotics)} words): ***\n{', '.join([t for t in itertools.islice(expanded_narcotics, 50)])}")
print(f"*** Expanded weapons ({len(expanded_weapons)} words): ***\n{', '.join([t for t in itertools.islice(expanded_weapons, 50)])}")
print(f"*** Expanded investigation ({len(expanded_investigation)} words) ***:\n{', '.join([t for t in itertools.islice(expanded_investigation, 50)])}")

*** Expanded narcotics (303 words): ***
anesthetic, drown, banning, pfizer, antipsychotic, prescribe, addict, tablets, amphetamine, grown, allergy, addiction, coroner, co, sophisticated, lace, od, steroid, antianxiety, methyl, psychotic, imitation, pseudoephedrine, bath, animal, acid, nonfatal, yerba, dose, pharmaceutical, cardiac, quantity, demerol, narcotic, induced, reservation, euphoric, hallucinogen, malarial, muscle, mari, paxil, addictive, meow, opiate, seconal, asphyxiation, hcl, danny, prescription
*** Expanded weapons (362 words): ***
guns, wielding, plated, hammer, mallet, ammunition, cannonball, gravity, imitation, unlicence, assult, semiautomatic, nine, iiia, degree, article, curve, indecent, warrior, wesson, paring, gage, volleys, cleaver, grab, enactments, vicious, cobra, powder, lascivious, short, silencer, steak, flash, breastplate, m, retractable, curved, pons, machete, ball, aggravated, unlawful, springfield, arrest, unlawfully, burglary, shotgun, lance, bodily
*** E

### Cleaning of not related words
Because of the underscore-splitting process (from compound words to singular words, e.g. 'new_amphetamine' becomes 'new' and 'amphetamine'; while 'amphetamine' is still a word of interest, 'new' is not), and because not all the GoogleVectors words are really meaningful in our topics of interest, it has been necessary to perform a manual cleaning process, going through the expanded seeds word-per-word, checking if its meaning was related to the corresponding topic of interest.
At the end of this process, 3 sets have been created, containing those words which are not related to the topics of interest but that were found during the expansion phase; these sets should be subtracted from the expanded sets in order to get the final seeds.

In [33]:
not_narcotics = {'suspect', 'medicine', 'gans', 'lab', 'dispensary', 'pain', 'sleeping', 'mouse', 'alliance', 'asphyxia', 'cheap', 'allergic', 'mrs', 
                 'arrest', 'sleep', 'drowning', 'vertex', 'contain', 'labs', 'heart', 'acute', 'buy', 'magic', 'pneumonia', 'usa', 'control', 'swallow', 
                 'abstracting', 'residue', 'treat', 'accidental', 'allergy', 'medications', 'dug', 'drown', 'cooks', 'nonfatal', 'quaid', 'grow', 'c', 
                 'salt', 'highly', 'traffic', 'medicinal', 'dope', 'decriminalize', 'medication', 'manufacture', 'reservation', 'precursor', 'plc', 'cook', 
                 'e', 'hepatitis', 'euphoric', 'boot', 'med', 'lace', 'lethal', 'twin', 'accidentally', 'generic', 'coroner', 'malarial', 'restraint', 'ba', 
                 'controlled', 'ultram', 'powerful', 'fit', 'mood', 'danny', 'illicit', 'toxicity', 'effects', 'commonly', 'his', 'quoad', 'n', 'legalize', 
                 'parkinson', 'fatal', 'amy', 'muscle', 'banning', 'sedentary', 'induce', 'electricity', 'patch', 'horse', 'smell', 'reaction', 'sophisticated', 
                 'ingest', 'cardiac', 'mice', 'bath', 'swallowed', 'anti', 'legalizing', 'altering', 'mmj', 'abstract', 'imitation', 'tar', 'unk', 'maker', 
                 'fatally', 'induced', 'medi', 'cocktail', 'surgical', 'grown', 'effect', 'patricia', 'co', 'ban', 'quantity', 'ya', 'clandestine', 'entertainer',
                 'new', 'animal', 'chemicals', 'storefront', 'pseudo', 'cr', 'containing', 'alias', 'newer', 'boots', 'alter', 'delirium', 'asphyxiation', 'gamma',
                 'wholesaler'}
not_weapons = {'addictive', 'bloodstaine', 'grievous', 'model', 'millimeter', 'instrument', 'plate', 'flame', 'shoplift', 'm', 'disorderly', 'kitchen', 
               'pastry', 'blender', 'plated', 'drug', 'pons', 'small', 'rape', 'power', 'battery', 'criminal', 'uttering', 'arrest', 'attempt', 'bread', 
               'bone', 'bar', 'cannabis', 'maggard', 'inch', 'fish', 'bloodstained', 'nonlethal', 'hang', 'blacksmith', 'unregistere', 'chuck', 'reckless',
               'control', 'polydrug', 'grevious', 'revolutionary', 'chucks', 'utter', 'chrome', 'using', 'crystal', 'treat', 'fillet', 'grab', 'tablets', 
               'lewd', 'banta', 'harassment', 'fix', 'heroin', 'sandal', 'opiate', 'unlicensed', 'semi', 'gravity', 'cause', 'article', 'salt', 'carrying',
               'recklessly', 'narcotic', 'highly', 'bodily', 'petit', 'dope', 'medicinal', 'prize', 'decriminalize', 'toothbrush', 'carving', 'resist',
               'tablet', 'threats', 'vicodin', 'pen', 'felonious', 'occasion', 'crow', 'liberties', 'inflict', 'wield', 'affray', 'automatic', 'charge', 
               'machine', 'acto', 'x', 'harm', 'light', 'concealed', 'lace', 'forcible', 'stimulant', 'springfield', 'prosecutor', 'cocaine', 
               'methamphetamine', 'contin', 'custodial', 'powered', 'altercation', 'trespassing', 'mm', 'theft', 'load', 'attempe', 'controlled', 'butcher',
               'attack', 'illicit', 'nun', 'steroid', 'brawl', 'possessing', 'double', 'facsimile', 'threat', 'nosed', 'legalize', 'mischief', 'enactment',
               'officer', 'liberty', 'fantasy', 'caleb', 'paring', 'short', 'meat', 'green', 'crack', 'tactical', 'incident', 'juana', 'fatal', 'unlawful',
               'vicious', 'intoxication', 'on', 'marihuana', 're', 'squad', 'banning', 'carry', 'civil', 'peacock', 'nine', 'peeler', 'inflicting', 'stick',
               'synthetic', 'pon', 'leather', 'drugs', 'trespass', 'conceal', 'ceremonial', 'conduct', 'cannabi', 'encampment', 'cultivation', 'flick', 
               'confinement', 'aggravated', 'homemade', 'cock', 'h', 'epic', 'painkiller', 'bath', 'amphetamines', 'flash', 'vegetable', 'sorcery', 'first', 
               'ornamental', 'shoplifting', 'sexual', 'scissor', 'mallet', 'fife', 'unlicence', 'retractable', 'malicious', 'imitation', 'amphetamine', 
               'wrench', 'substances', 'steak', 'mighty', 'substance', 'burglary', 'centimeter', 'brittany', 'serpent', 'stanley', 'narcotics', 'causing', 
               'endangerment', 'addiction', 'sig', 'unlawfully', 'marijuana', 'bugle', 'ons', 'balls', 'forge', 'assualt', 'domestic', 'felony', 'resisting', 
               'curve', 'ball', 'unlicense', 'pare', 'stimulants', 'methadone', 'caine', 'ban', 'concealable', 'class', 'co', 'deadly', 'meth', 'prescription', 
               'sault', 'murder', 'allegedly', 'unprovoked', 'indecent', 'filet', 'box', 'hallucinogen', 'potent', 'era', 'cobra', 'robbery', 'unregistered', 
               'use', 'war', 'mari', 'molestation', 'salute', 'lascivious', 'incense', 'threatening', 'threaten', 'degree', 'curved', 'phencyclidine', 
               'possess','prescribe', 'warrior', 'abercrombie', 'bowie', 'prescribing', 'misdemeanor', 'fixed', 'carve'}
not_investigation = {'cache', 'drink', 'mask', 'scrap', 'id', 'protective', 'rest', 'surrey', 'prison', 'heart', 'brega', 'firearm', 'bar', 'arousal',
                     'gunman', 'touch', 'accuse', 'handgun', 'embezzlement', 'psychopath', 'utter', 'misdemeanants', 'name', 'shooters', 'pizza', 
                     'truancy', 'mara', 'abortion', 'relapse', 'traffic', 'hate', 'carlo', 'incest', 'concealing', 'jewelry', 'wield', 'sheet', 
                     'joshua', 'organize', 'task', 'dotson', 'prosecutor', 'parental', 'psychopathic', 'daylight', 'banger', 'magistrate', 'offenses',
                     'detaining', 'milly', 'nicola', 'attack', 'wielding', 'santos', 'parolees', 'facilities', 'grand', 'organized', 'foxen', 
                     'postpone', 'hillside', 'in', 'arpaio', 'civilly', 'whereabouts', 'hilton', 'calculating', 'proceedings', 'reoffende', 'illinois',
                     'fedex', 'anton', 'bars', 'wnt', 'markus', 'cobras', 'debit', 'mccarthy', 'attacks', 'behavior', 'reputed', 'tool', 'ralph', 
                     'subway', 'standoff', 'apts', 'bag', 'tamper', 'patronizing', 'ted', 'fake', 'felony', 'oakdale', 'brotherhood', 'ms', 'scheme',
                     'attorney', 'night', 'moat', 'unprotected', 'bully', 'mathew', 'selby', 'rates', 'bail', 'stab', 'indecent', 'cell', 'kiddie',
                     'detention', 'w', 'releasing', 'motorcycle', 'detainer', 'correctional', 'protection', 'angels', 'los', 'shipman', 'activity',
                     'supremacist', 'brother', 'armed', 'pistol', 'grasso', 'jury', 'baseline', 'white', 'mysterious', 'youthful', 'naming', 'caseload',
                     'brazen', 'uninsurance', 'reconviction', 'response', 'tampering', 'knife', 'bali', 'aces', 'botched', 'woode', 'buy', 'sandwich',
                     'solido', 'tools', 'harold', 'facility', 'mentally', 'nonviolent', 'lewd', 'ace', 'plead', 'car', 'parole', 'break', 'petit', 
                     'offense', 'safe', 'marc', 'pantry', 'similar', 'pornography', 'carlton', 'classic', 'richard', 'johnny', 'justice', 'stole', 
                     'spiking', 'spear', 'forcible', 'juvenile', 'knowledge', 'home', 'imprison', 'jigsaw', 'pink', 'repeat', 'mischief', 'bungle', 
                     'norinco', 'sam', 'petty', 'siders', 'expert', 'squad', 'scenery', 'joe', 'artist', 'prolific', 'chew', 'refuse', 'conceal', 
                     'habitual', 'ins', 'trespass', 'queen', 'advanced', 'reincarceration', 'distraction', 'postponed', 'misconduct', 'calabrian', 
                     'cop', 'prisoner', 'ram', 'imperial', 'infection', 'blue', 'jeremy', 'black', 'computer', 'handbag', 'detroit', 'chucky', 
                     'brandish', 'ice', 'incarceration', 'security', 'identity', 'mai', 'antoni', 'prisons', 'prosecution', 'misappropriation',
                     'scare', 'cobra', 'mexican', 'teen', 'notorious', 'tin', 'levi', 'walter', 'sider', 'v', 'crime', 'masked', 'dowler', 'aryan',
                     'misdemeanor', 'torrio', 'emergency', 'handcuff', 'detachment', 'human', 'debs', 'mickey', 'mandatory', 'defilement', 'instrument',
                     'agency', 'hilltop', 'drug', 'deportation', 'zodiac', 'efrain', 'cold', 'hobo', 'naples', 'spousal', 'uttering', 'dui', 'targeted',
                     'attempt', 'detain', 'exploitation', 'incarcerated', 'unregistere', 'neighborhood', 'reconvict', 'moor', 'mortality', 'harbor',
                     'meditate', 'testify', 'scared', 'hiv', 'ee', 'sheets', 'disposition', 'joanna', 'refused', 'virus', 'wedlock', 'ex', 'caller',
                     'reentry', 'statutory', 'lords', 'exposure', 'deputy', 'convict', 'confined', 'money', 'metal', 'trojan', 'operation', 'reduce',
                     'angel', 'collar', 'hospitalization', 'apt', 'rate', 'bust', 'threat', 'straight', 'custody', 'liberty', 'card', 'dr', 'vito', 
                     'purse', 'sicilian', 'guilty', 'doll', 'jail', 'serial', 'fatal', 'revocation', 'alleged', 're', 'spike', 'return', 'recognizance',
                     'variant', 'deuces', 'accused', 'capias', 'nonforcible', 'do', 'nathan', 'pregnancy', 'conviction', 'malicious', 'lover', 
                     'substance', 'misdemeanant', 'psychiatric', 'persistent', 'melbourne', 'welfare', 'syndicate', 'detainment', 'pietro', 'graffiti',
                     'crimes', 'winkler', 'raoul', 'infringement', 'infant', 'doc', 'own', 'victim', 'ar', 'street', 'krishna', 'bundy', 'calculate', 
                     'indecently', 'within', 'rested', 'forcibly', 'lynn', 'net', 'harden', 'ill', 'worm', 'neapolitan', 'path', 'carnal', 'lab', 
                     'langner', 'daytime', 'whereabout', 'tony', 'bamber', 'clear', 'odometer', 'criminal', 'arresting', 'bandit', 'birney', 'reducing',
                     'bandana', 'rascal', 'lonely', 'bredel', 'ira', 'digital', 'tire', 'uniform', 'baron', 'biker', 'oo', 'cleared', 'target', 'touching',
                     'queens', 'indictment', 'pre', 'allege', 'slashing', 'destruction', 'patronize', 'child', 'apprehend', 'crush', 'enzyme', 'chinese',
                     'charge', 'thirst', 'offend', 'policing', 'moh', 'force', 'probation', 'overcrowd', 'teodor', 'custodial', 'pect', 'prevalence',
                     'dropout', 'teenage', 'arraignment', 'gay', 'shoot', 'yorkshire', 'bike', 'u', 'insane', 'victimization', 'tiny', 'store', 'incident',
                     'baby', 'minimum', 'antisocial', 'mass', 'family', 'project', 'offender', 'la', 'capia', 'rearreste', 'waive', 'hardened', 'mad', 
                     'confine', 'converter', 'centers', 'gord', 'sting', 'domain', 'german', 'man', 'deuce', 'deliveryman', 'detection', 'release', 'police',
                     'catalytic', 'proceeding', 'dump', 'nostra', 't', 'surrender', 'apartment', 'center', 'behind', 'meth', 'bullying', 'batterer', 'felon',
                     'willful', 'birth', 'genovese', 'consensual', 'converters', 'wooded', 'overcrowded', 'unregistered', 'reconvicted', 'zeus', 'knifepoint',
                     'apprehension', 'vigilante', 'brutally', 'upstart'}

In [34]:
final_narcotics = expanded_narcotics - not_narcotics
final_weapons = expanded_weapons - not_weapons
final_investigation = expanded_investigation - not_investigation

In [35]:
print(f"*** Final narcotics ({len(final_narcotics)} words): ***\n{', '.join([t for t in itertools.islice(final_narcotics, 50)])}")
print(f"*** Final weapons ({len(final_weapons)} words): ***\n{', '.join([t for t in itertools.islice(final_weapons, 50)])}")
print(f"*** Final investigation ({len(final_investigation)} words) ***:\n{', '.join([t for t in itertools.islice(final_investigation, 50)])}")

*** Final narcotics (167 words): ***
anesthetic, pfizer, antipsychotic, prescribe, addict, tablets, amphetamine, addiction, od, steroid, antianxiety, methyl, psychotic, pseudoephedrine, acid, yerba, dose, pharmaceutical, demerol, narcotic, hallucinogen, mari, addictive, paxil, opiate, meow, seconal, hcl, prescription, paraphenalia, methaqualone, lorazepam, sodium, alia, librium, amytal, phenytoin, juana, sativa, codeine, cultivate, phencyclidine, powder, ativan, clonazepam, chemical, diphenhydramine, caine, acetaminophen, banta
*** Final weapons (158 words): ***
guns, gun, sk, wielding, armed, barreled, armored, hammer, cannonball, ammunition, colt, carbine, loader, chainsaw, barrel, sks, nuclear, bayonet, weap, saw, assult, tec, scythe, flintlock, fire, shank, armor, dagger, pellet, semiautomatic, howitzer, breech, musket, iiia, knife, gatle, firing, thompson, scar, luger, loaded, wesson, stabbing, claw, cutlass, gage, volleys, bladed, nova, cleaver
*** Final investigation (245 words)

In [36]:
_ = check_corpus_presence(final_narcotics, final_weapons, final_investigation, tokens, topN=None, notFoundN=None)

(1, 'absconder'), (1, 'affiliations'), (1, 'ambien'), (1, 'amitriptyline'), (1, 'amphetamines'), (1, 'antidepression'), (1, 'antiretroviral'), (1, 'arrrest'), (1, 'assualt'), (1, 'assult'), (1, 'blunderbuss'), (1, 'breastplate'), (1, 'chainsaw'), (1, 'clonazepam'), (1, 'criminals'), (1, 'crip'), (1, 'crooks'), (1, 'cyber'), (1, 'dextromethorphan'), (1, 'diazepam'), (1, 'familia'), (1, 'felons'), (1, 'flintlock'), (1, 'gangbanger'), (1, 'gatle'), (1, 'harasser'), (1, 'hcl'), (1, 'herion'), (1, 'homocide'), (1, 'howitzer'), (1, 'indictments'), (1, 'insider'), (1, 'ketamine'), (1, 'killings'), (1, 'larcency'), (1, 'lorazepam'), (1, 'manhunt'), (1, 'meow'), (1, 'minie'), (1, 'minimi'), (1, 'molestor'), (1, 'moors'), (1, 'murderers'), (1, 'neonaticide'), (1, 'painkiller'), (1, 'paxil'), (1, 'phenytoin'), (1, 'pills'), (1, 'polydrug'), (1, 'preying'), (1, 'profiler'), (1, 'propoxyphene'), (1, 'psychedelic'), (1, 'raping'), (1, 'reoffenders'), (1, 'reoffense'), (1, 'robbed'), (1, 'sabers'), (

In [37]:
_ = check_corpus_presence(not_narcotics, not_weapons, not_investigation, tokens)  # Check if there is something borderline which appear lot of times that should be considered as interesting

(1, 'aces'), (1, 'amphetamines'), (1, 'assualt'), (1, 'attacks'), (1, 'bali'), (1, 'balls'), (1, 'batterer'), (1, 'boots'), (1, 'botched'), (1, 'bredel'), (1, 'bugle'), (1, 'bullying'), (1, 'bungle'), (1, 'calabrian'), (1, 'chucks'), (1, 'chucky'), (1, 'cleared'), (1, 'concealed'), (1, 'cooks'), (1, 'dug'), (1, 'effects'), (1, 'epic'), (1, 'foxen'), (1, 'genovese'), (1, 'grasso'), (1, 'hilltop'), (1, 'id'), (1, 'kiddie'), (1, 'krishna'), (1, 'langner'), (1, 'legalizing'), (1, 'mice'), (1, 'milly'), (1, 'mmj'), (1, 'nonlethal'), (1, 'norinco'), (1, 'nostra'), (1, 'ons'), (1, 'painkiller'), (1, 'parolees'), (1, 'polydrug'), (1, 'pons'), (1, 'postponed'), (1, 'reconvicted'), (1, 'serpent'), (1, 'siders'), (1, 'sorcery'), (1, 'stimulants'), (1, 'swallowed'), (1, 'threats'), (1, 'torrio'), (1, 'ultram'), (1, 'uninsurance'), (1, 'unlicence'), (1, 'upstart'), (2, 'abstracting'), (2, 'acto'), (2, 'angels'), (2, 'apts'), (2, 'bamber'), (2, 'catalytic'), (2, 'cobras'), (2, 'concealable'), (2, 'c

## Save to local memory

In [38]:
from src.seeds import Seeds

In [39]:
seeds = Seeds()

In [40]:
seeds.save_word2vec_seeds(new_narcotics, new_weapons, new_investigation)
seeds.save_word2vec_lemmatized_seeds(new_narcotics_lemma, new_weapons_lemma, new_investigation_lemma)
seeds.save_word2vec_combined_seeds(expanded_narcotics, expanded_weapons, expanded_investigation)
seeds.save_final_filtered_seeds(final_narcotics, final_weapons, final_investigation)

In [41]:
n, w, i = seeds.get_final_filtered_seeds()
print(len(n), len(w), len(i))

167 158 245
